# Relevant imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import math
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

import datetime

from autocorrect import Speller

from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

from datetime import date

import geopandas as gpd
from shapely.geometry import shape, Point

import warnings
warnings.filterwarnings('ignore')

# I. Dataset(s) preparation and cleaning

Before we proceed to tackle each of our research questions, some data cleaning is in order.

## 1. Load the data and explore its structure

In [2]:
inspections = pd.read_csv('datasets/food-inspections.csv')

In [3]:
len(inspections)

195736

The dataset has 22 columns. Let's examine what each of them is.

In [4]:
#Display columns
inspections.columns

Index(['Inspection ID', 'DBA Name', 'AKA Name', 'License #', 'Facility Type',
       'Risk', 'Address', 'City', 'State', 'Zip', 'Inspection Date',
       'Inspection Type', 'Results', 'Violations', 'Latitude', 'Longitude',
       'Location', 'Historical Wards 2003-2015', 'Zip Codes',
       'Community Areas', 'Census Tracts', 'Wards'],
      dtype='object')

In [5]:
inspections.dtypes

Inspection ID                   int64
DBA Name                       object
AKA Name                       object
License #                     float64
Facility Type                  object
Risk                           object
Address                        object
City                           object
State                          object
Zip                           float64
Inspection Date                object
Inspection Type                object
Results                        object
Violations                     object
Latitude                      float64
Longitude                     float64
Location                       object
Historical Wards 2003-2015    float64
Zip Codes                     float64
Community Areas               float64
Census Tracts                 float64
Wards                         float64
dtype: object

A description of the features is given below [Source](https://data.cityofchicago.org/api/assets/BAD5301B-681A-4202-9D25-51B2CAE672FF).
The last five columns are ignored in the dataset source; we will see that those columns are in fact null

| Feature name                | Variable Type | Description 
|-----------------------------|---------------|--------------------------------------------------------
| Inspection ID        | Integer    | The inspection unique identifier.
| DBA Name                 | String        | ‘Doing business as.’Legal name of the establishment.
| AKA NAme                | String    |  ‘Also known as.’ Name the public would know the establishment as.
| License # | Integer    | Unique number assigned to the establishment for the purposes of licensing by the Department of Business Affairs and Consumer Protection.
| Type of facility                | String    | Each establishment is described by one of the following: bakery, banquet hall, candy store, caterer, coffee shop, day care center (for ages less than 2), day care center (for ages 2 – 6), day care center (combo, for ages less than 2 and 2 – 6 combined), gas station, Golden Diner, grocery store, hospital, long term care center(nursing home), liquor store, mobile food dispenser, restaurant, paleteria, school, shelter, tavern, social club, wholesaler, or Wrigley Field Rooftop.
| Risk                   | String    | Risk category of facility of adversely affecting the public’s health, with 1 being the highest and 3 the lowest. The frequency of inspection is tied to this risk, with risk 1 establishments inspected most frequently and risk 3 least frequently.
| Address        | String    | Street address of the establishment.
| City        | String    | City of the establishment.
| State        | String    | State of the establishment.
| Zip        | Integer    | Zip code of the establishment.
| Inspection Date        | Date    | Date of the inspection
| Inspection Type        | String    | An inspection can be one of the following types: canvass, the most common type of inspection performed at a frequency relative to the risk of the establishment; consultation, when the inspection is done at the request of the owner prior to the opening of the establishment; complaint, when the inspection is done in response to a complaint against the establishment; license, when the inspection is done as a requirement for the establishment to receive its license to operate; suspect food poisoning, when the inspection is done in response to one or more persons claiming to have gotten ill as a result of eating at the establishment (a specific type of complaint-based inspection); task-force inspection, when an inspection of a bar or tavern is done. Re-inspections can occur for most types of these inspections and are indicated as such.
| Results        | String    | Results: An inspection can pass, pass with conditions or fail. Establishments receiving a ‘pass’ were found to have no critical or serious violations (violation number 1-14 and 15- 29, respectively). Establishments receiving a ‘pass with conditions’ were found to have critical or serious violations, but these were corrected during the inspection. Establishments receiving a ‘fail’ were found to have critical or serious violations that were not correctable during the inspection. An establishment receiving a ‘fail’ does not necessarily mean the establishment’s licensed is suspended. Establishments found to be out of business or not located are indicated as such.
| Violations        | String    | An establishment can receive one or more of 45 distinct violations (violation numbers 1-44 and 70). For each violation number listed for a given establishment, the requirement the establishment must meet in order for it to NOT receive a violation is noted, followed by a specific description of the findings that caused the violation to be issued.
| Latitude        | Integer    | Latitude of the establishment.
| Longitude        | Integer    | Longitude of the establishment.



## 2. Drop duplicates

The dataset source explicitly says there are duplicates in our data, hence it makes sence to drop those. [source](https://www.kaggle.com/chicago/chicago-food-inspections)

In [6]:
inspections.drop_duplicates(inplace=True)
len(inspections)

195524

## 3. Dataset cleaning

### A. Drop null columns

The 'Location' column contains the latitude and longitude of the establishment. However, there are separate 'Latitude' and 'Longitude' columns. We can hence safely drop the 'Location' column.

In [7]:
inspections = inspections.drop(columns=['Location'])

The head of the dataset only contains NaN entries for the 'Historical Wards 2003-2015', 'Zip Codes', 'Community Areas', 'Census Tracts', 'Wards' columns. Let's see if this is true for the whole dataset.

In [8]:
# make sure that our assumption is correct
print('Values taken by \'Historical Wards 2003-2015\': ', inspections['Zip Codes'].unique())
print('Values taken by \'Zip Codes\': ', inspections['Zip Codes'].unique())
print('Values taken by \'Community Areas\': ', inspections['Zip Codes'].unique())
print('Values taken by \'Census Tracts\': ', inspections['Zip Codes'].unique())
print('Values taken by \'Wards\': ', inspections['Zip Codes'].unique())


Values taken by 'Historical Wards 2003-2015':  [nan]
Values taken by 'Zip Codes':  [nan]
Values taken by 'Community Areas':  [nan]
Values taken by 'Census Tracts':  [nan]
Values taken by 'Wards':  [nan]


We drop all columns apart from the 'Community Areas' because we will be needing it in our study. We will fill later.

In [9]:
inspections = inspections.drop(columns=['Historical Wards 2003-2015', 'Zip Codes', 'Census Tracts', 'Wards'])

### B. Clean the location related features and fill in community area feature

Let's examine if the whole dataset is relevent to the study we are conducting by seeing which entries correspond to facilities in Chicago.

First, we check if there are any missing values for the column 'City' or 'State'

In [10]:
#Investigate the state=nan and city=nan restaurants
inspections[pd.isnull(inspections.State) | pd.isnull(inspections.City)].head()

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Community Areas
1669,2312774,CHICAGO COLLEGIATE CHARTER,CHICAGO COLLEGIATE CHARTER,3846104.0,School,Risk 1 (High),10909 S COTTAGE GROVE AVE,NaN,IL,NaN,2019-09-24T00:00:00.000,Canvass Re-Inspection,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.696087,-87.608945,NaN
1879,2312540,CHICAGO COLLEGIATE CHARTER,CHICAGO COLLEGIATE CHARTER,3846104.0,School,Risk 1 (High),10909 S COTTAGE GROVE AVE,NaN,IL,NaN,2019-09-19T00:00:00.000,Canvass Re-Inspection,Fail,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.696087,-87.608945,NaN
1903,2312545,JCYS IRIS & STEVEN PODOLSKY FAMILY CENTER,JCYS IRIS & STEVEN PODOLSKY FAMILY CENTER,2671297.0,Children's Services Facility,Risk 1 (High),2112 W LAWRENCE AVE,NaN,IL,60625.0,2019-09-19T00:00:00.000,License Re-Inspection,Pass,"38. INSECTS, RODENTS, & ANIMALS NOT PRESENT - ...",41.968821,-87.682201,NaN
3073,2305166,"AMY BECK CAKE DESIGN, LLC","AMY BECK CAKE DESIGN, LLC",2079264.0,Bakery,Risk 1 (High),636 N RACINE AVE,NaN,NaN,60642.0,2019-08-23T00:00:00.000,Canvass,Pass,"55. PHYSICAL FACILITIES INSTALLED, MAINTAINED ...",41.893380,-87.657588,NaN
3617,2304583,JCYS IRIS & STEVEN PODOLSKY FAMILY CENTER,JCYS IRIS & STEVEN PODOLSKY FAMILY CENTER,2671297.0,Children's Services Facility,Risk 1 (High),2112 W LAWRENCE AVE,NaN,IL,60625.0,2019-08-13T00:00:00.000,License,Fail,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.968821,-87.682201,NaN


Looking at the coordinates of these places, all of them seem to also be in chicago, so we will fill their City and State columns

In [11]:
inspections['City'] = inspections['City'].fillna('Chicago')
inspections['State'] = inspections['State'].fillna('IL')

Next, we check if there are any facilities which are not located in Chicago.

In [12]:
# make sure that our assumption is correct
print('Values taken by \'City\': ', inspections['City'].unique())

Values taken by 'City':  ['CHICAGO' 'Chicago' 'chicago' 'GRIFFITH' 'NEW YORK' 'SCHAUMBURG'
 'ELMHURST' 'ALGONQUIN' 'NEW HOLSTEIN' 'CCHICAGO' 'NILES NILES' 'EVANSTON'
 'CHICAGO.' 'CHESTNUT STREET' 'LANSING' 'CHICAGOCHICAGO' 'WADSWORTH'
 'WILMETTE' 'WHEATON' 'CHICAGOHICAGO' 'ROSEMONT' 'CHicago' 'CALUMET CITY'
 'PLAINFIELD' 'HIGHLAND PARK' 'PALOS PARK' 'ELK GROVE VILLAGE' 'CICERO'
 'BRIDGEVIEW' 'OAK PARK' 'MAYWOOD' 'LAKE BLUFF' '312CHICAGO'
 'SCHILLER PARK' 'SKOKIE' 'BEDFORD PARK' 'BANNOCKBURNDEERFIELD' 'CHCICAGO'
 'BLOOMINGDALE' 'Norridge' 'CHARLES A HAYES' 'CHCHICAGO' 'CHICAGOI'
 'SUMMIT' 'OOLYMPIA FIELDS' 'WESTMONT' 'CHICAGO HEIGHTS' 'JUSTICE'
 'TINLEY PARK' 'LOMBARD' 'EAST HAZEL CREST' 'COUNTRY CLUB HILLS'
 'STREAMWOOD' 'BOLINGBROOK' 'INACTIVE' 'BERWYN' 'BURNHAM' 'DES PLAINES'
 'LAKE ZURICH' 'OLYMPIA FIELDS' 'alsip' 'OAK LAWN' 'BLUE ISLAND' 'GLENCOE'
 'FRANKFORT' 'NAPERVILLE' 'BROADVIEW' 'WORTH' 'Maywood' 'ALSIP'
 'EVERGREEN PARK']


We can see that this column takes values which are not Chicago. The rows where the 'City' is not Chicago are hence irrelevent to our study and should be dropped. Let's first make sure tha the bulk of the data is for Chicago before proceeding

In [13]:
chicago_inspections = inspections.groupby('City')['Inspection ID'].nunique().filter(regex='(?i)chicago', axis=0)
print('{}% of the inpections in the dataframe come from Chicago.'.format(100 * chicago_inspections.values.sum()/len(inspections)))

99.89668787463431% of the inpections in the dataframe come from Chicago.


We can safely drop the rows which come from cities that are not Chicago.

In [14]:
# list of ways Chicago has been written in the dataset
chicago_variations = chicago_inspections.index.tolist()
inspections = inspections[inspections['City'].isin(chicago_variations)]
# drop the 'City' and 'State' columns since they have each only one value, 'Chicago' and 'IL' respectively
inspections = inspections.drop(columns=['City', 'State'])

Now that we only have facilities in Chicago in our dataset, let us fill the 'Community Areas' column. To that end, we use the geopy library.

We start by getting the unique locations in the dataset.

In [15]:
def combineloc(latitude, longitude):
    """
    function to format the latitude and longitude such that they can be used in geopy requests
    """
    return '{}, {}'.format(latitude, longitude)

In [16]:
locations = inspections['Latitude'].dropna().combine(inspections['Longitude'].dropna(),combineloc)
unique_locs = locations.unique()

In [17]:
unique_locs

array(['41.92799528871574, -87.78575236468352',
       '41.946140053442825, -87.73518301995274',
       '41.93592957402078, -87.64440716256712', ...,
       '41.764896400247046, -87.65396483351302',
       '41.768328334800714, -87.67381938402686',
       '41.846516428599394, -87.69542345938575'], dtype=object)

In [18]:
len(unique_locs)

16812

In [19]:
unique_locs_s = pd.Series(unique_locs, dtype=str)

We then request the geopy entry for the locations we have (code takes 4h40 to run as we can only do one geopy query per second) and save the areas in a pickle.

In [20]:
# geolocator = Nominatim(timeout=17000,user_agent="area_filler")
# geocode = RateLimiter(geolocator.reverse, min_delay_seconds=1)
# # for i in unique_locs:
# #     print(i)
# #     print(geolocator.reverse(i))
# areas = unique_locs_s.copy().apply(geocode)
# areas.to_pickle('./areas')

In [21]:
areas = pd.read_pickle('./areas.pickle')

In [22]:
areas.isna().sum()

0

Let's add the community areas and neighborhoods to the dataframe.

In [23]:
# get latitude, longitude and corresponding community area and neighbourhood in same dataframe
suburbs_neighbourhoods = [(x.raw.get('address', {}).get('suburb',np.nan), x.raw.get('address', {}).get('neighbourhood',np.nan)) for x in areas]
suburbs, neighbourhoods = zip(*suburbs_neighbourhoods)
locs_df = pd.concat([pd.Series(unique_locs, name='Location'), pd.Series(suburbs,name='Community Area'), pd.Series(neighbourhoods,name='Neighbourhood')], axis=1)

In [24]:
# add the community area and the neighbourhood to each entry in our dataframe
inspections['Location'] = inspections['Latitude'].combine(inspections['Longitude'],combineloc)
inspections = inspections.merge(locs_df,on='Location',how='outer')
inspections = inspections.drop(columns=['Community Areas'])

Let's check if there are any NaN entries in our 'Community Area' column

In [25]:
print('{}% of rows have missing Community Areas'.format(100 *(inspections['Community Area'].isna().sum()/len(inspections))))
print('{} total number of missing Community Areas'.format(inspections['Community Area'].isnull().sum()))

3.192164733107382% of rows have missing Community Areas
6235 total number of missing Community Areas


We see that geopy doesn't return the community area for 6235 locations, if we do a simple Google search, we notice that most of these locations are located in the same community areas (notably JEFFERSON PARK, WEST ELSDON, BRIGHTON PARK, EAST SIDE, ASHBURN).

**Hence in order to fill in those missing values, we introduce a dataset containing the different boundaries of the community areas in Chicago as multipolygon objects. The data set is available under the City of Chicago data portal. [source](https://www.chicago.gov/city/en/depts/doit/dataset/boundaries_-_communityareas.html)** (we download the GeoJSON file extension format)

We process the GEOdataframe in a way that is useful to our analysis and then export it as a new .geojson file that we use for the rest of the data cleaning.

In [26]:
# set the filepath and load in a shapefile
#fp = "./datasets/boundaries.geojson"
#boundaries_community_areas = gpd.read_file(fp)
# check data type so we can see that this is not a normal dataframe, but a GEOdataframe
#boundaries_community_areas.head()

In [27]:
#boundaries_community_areas=boundaries_community_areas[['community','geometry']]
#boundaries_community_areas.loc[74,'community']='O\'HARE'
#boundaries_community_areas.sort_values(by=['community'], inplace=True)
#boundaries_community_areas.reset_index(inplace=True,drop=True)
#boundaries_community_areas.reset_index(inplace=True,drop=False)
#boundaries_community_areas.rename(columns={"index": "area_number"},inplace=True)
#boundaries_community_areas['community'] = boundaries_community_areas['community'].astype('str')
#boundaries_community_areas.to_file("boundaries_processed.geojson", driver="GeoJSON")

In [28]:
# Read the cleaned data and proceed to replace the missing community area values
fp = "./boundaries_processed.geojson"
boundaries_community_areas = gpd.read_file(fp)
boundaries_community_areas.head()

,area_number,community,geometry
0,0,ALBANY PARK,"MULTIPOLYGON (((-87.70404 41.97355, -87.70403 ..."
1,1,ARCHER HEIGHTS,"MULTIPOLYGON (((-87.71437 41.82604, -87.71436 ..."
2,2,ARMOUR SQUARE,"MULTIPOLYGON (((-87.62917 41.84556, -87.62947 ..."
3,3,ASHBURN,"MULTIPOLYGON (((-87.71255 41.75734, -87.71252 ..."
4,4,AUBURN GRESHAM,"MULTIPOLYGON (((-87.63990 41.75615, -87.63990 ..."


We do a sanity check to see if the community areas in the inspection data we got from geopy correspond to the community areas we get from geopy.

In [29]:
# Preprocess the boundaries we have
inspections['Community Area'] = inspections['Community Area'].str.upper().str.strip()
areas_geopy = set(inspections['Community Area'].drop_duplicates())

In [30]:
missing = set(boundaries_community_areas['community'])-areas_geopy
missing = pd.DataFrame(missing, columns=['Community Area'])
print("Community areas missing after getting community areas from geopy: ")
missing

Community areas missing after getting community areas from geopy: 


,Community Area
0,WEST ELSDON
1,EAST SIDE
2,JEFFERSON PARK
3,BRIGHTON PARK
4,ASHBURN


Let's use the multipolygon objects representing the community areas to verify using the location of a restaurant if it's contained in that community area and fill in the missing values.

In [31]:
def find_community_area(establishment):
    point = Point(establishment['Longitude'], establishment['Latitude'])
    for i,multipolygon in enumerate(boundaries_community_areas['geometry']):
        for polygon in list(multipolygon.geoms):
            if point.within(polygon):
                return boundaries_community_areas.loc[i]['community']
    return np.NaN


missing_community_areas = inspections.loc[inspections['Community Area'].isnull()]
for i in missing_community_areas.index:
    missing_community_areas.loc[i,('Community Area')] = find_community_area(missing_community_areas.loc[i])

In [32]:
inspections.loc[inspections['Community Area'].isnull()] = missing_community_areas
inspections['Community Area'].isnull().sum()

519

We see that we could recover more than 90% of the missing community areas using this approch and we can safely discars the establishments we still can't triangulate because they are likely not located in Chicago :D !

In [33]:
inspections = inspections[inspections['Community Area'].notna()]

Let's now examine if everything checks out

In [34]:
areas_geopy = inspections['Community Area'].isnull()

In [35]:
areas_geopy = inspections['Community Area'].unique()
missing = set(areas_geopy)-set(boundaries_community_areas['community'])
missing = pd.DataFrame(missing, columns=['Community Area'])
print("Community areas missing in the boundaries dataset: ")
missing

Community areas missing in the boundaries dataset: 


,Community Area
0,PILSEN
1,CHINATOWN
2,ILLINOIS MEDICAL DISTRICT


We see that CHINATOWN, ILLINOIS MEDICAL DISTRICT, and PILSEN are mentioned in our dataset but not in the community areas boundries dataset.

After an easy Google search we see that:
* CHINATOWN is in reality ARMOUR SQUARE
* ILLINOIS MEDICAL DISTRICT is part of NEAR WEST SIDE
* PILSEN is part of LOWER WEST SIDE

Hence we change those community areas in our inspection dataset.

In [36]:
community_dict = {"Community Area" : {"CHINATOWN": "ARMOUR SQUARE",
                   "ILLINOIS MEDICAL DISTRICT": "NEAR WEST SIDE",
                    "PILSEN": "LOWER WEST SIDE"}}
inspections.replace(community_dict, inplace=True)

### C. Check which columns still have missing values

Let's check if there are anymore missing values in the dataframe.

In [37]:
inspections.isna().sum().apply(lambda x: '{}% missing values'.format(100 * x/len(inspections)))

Inspection ID                       0.0% missing values
DBA Name                            0.0% missing values
AKA Name              1.247414054198344% missing values
License #          0.008726764988218868% missing values
Facility Type         2.444007535818237% missing values
Risk                0.03644707730373762% missing values
Address                             0.0% missing values
Zip                  0.0241269384968404% missing values
Inspection Date                     0.0% missing values
Inspection Type    0.000513339116954051% missing values
Results                             0.0% missing values
Violations           26.520125460080184% missing values
Latitude                            0.0% missing values
Longitude                           0.0% missing values
Location                            0.0% missing values
Community Area                      0.0% missing values
Neighbourhood         16.56750666057504% missing values
dtype: object

In [38]:
missing_inspections = inspections[inspections['Inspection ID'].isnull()]
print("Number of missing inspections ID: ", len(missing_inspections))

Number of missing inspections ID:  0


In [39]:
inspections.dropna(subset=['Inspection ID'],inplace = True)

In [40]:
inspections.isna().sum().apply(lambda x: '{}% missing values'.format(100 * x/len(inspections)))

Inspection ID                       0.0% missing values
DBA Name                            0.0% missing values
AKA Name              1.247414054198344% missing values
License #          0.008726764988218868% missing values
Facility Type         2.444007535818237% missing values
Risk                0.03644707730373762% missing values
Address                             0.0% missing values
Zip                  0.0241269384968404% missing values
Inspection Date                     0.0% missing values
Inspection Type    0.000513339116954051% missing values
Results                             0.0% missing values
Violations           26.520125460080184% missing values
Latitude                            0.0% missing values
Longitude                           0.0% missing values
Location                            0.0% missing values
Community Area                      0.0% missing values
Neighbourhood         16.56750666057504% missing values
dtype: object

* The AKA names still have missing entries. We will replace those with the DBA name because we will need those for our recommendation map later on, and it make more sense to display the AKA names for the users. However will be  mostly sticking to the DBA Name when referring to establishments.
* The Lisence Number is missing for some entries. Seeing as it is not essential in our main analysis we will not pay attention to it for now.
* The missing Zip entries are not important as we have enough information regarding location (latitude, longitude, community area and address). Hence we can safely drop this column.
* The number of missing neighbourhoods is quite big. Hence, we may drop that column as well.
* We will try to recover the missing facility type from the restaurant's name using other entries where the name is the same and the type is filled in.
* We will see if the missing violations entries are consistent and have something to do with the inspection type and inspection results.
* The fraction of the data with missing Inspection Type represent a really small fraction in our dataset. Hence we can safely drop those entries.
* Missing values of Risk is the only ones that might hinder our analysis. We will try to recover those using the restaurant's name and other filled in entries. For the values that we cannot recover, we drop the corresponding entries.

### D. Drop unneeded columns

In [41]:
# drop neighbourhood and zip columns
inspections = inspections.drop(columns=['Neighbourhood','Zip'])

### E. Clean Facility Type column

We first examine the facility type entries.

In [42]:
inspections['Facility Type'].unique()

array(['Restaurant', nan, 'Grocery Store', "Children's Services Facility",
       'COFFEE SHOP', 'Liquor', 'Catering', 'Daycare (2 - 6 Years)',
       'Daycare Above and Under 2 Years', 'Bakery', 'Airport Lounge',
       "O'Hare Kiosk", 'Lounge', 'STORE', 'BEVERAGE/SILVERWARE WAREHOUSE',
       'VENDING MACHINE', 'KIOSK', 'Mobile Food Dispenser',
       'TEMPORARY KIOSK', 'School',
       'Pop-Up Food Establishment User-Tier II',
       '15 monts to 5 years old', 'Daycare Combo 1586', 'LIVE POULTRY',
       'CONVENIENCE', 'GAS STATION/GROCERY',
       'GAS STATION/CONVENIENCE STORE', 'Daycare (Under 2 Years)',
       'COFFEE CART', 'Day Care Combo (1586)', 'CHARTER SCHOOL',
       "1023 CHILDERN'S SERVICE S FACILITY", 'Long Term Care',
       'LIQUOR/COFFEE KIOSK', 'Mobile Food Preparer', 'MOBILE FOOD',
       'Mobile Prepared Food Vendor', 'Shared Kitchen User (Long Term)',
       'Wholesale', 'Pop-Up Establishment Host-Tier II',
       'TEACHING SCHOOL', 'CONVENIENCE STORE', 'FRENCH 

We notice that some facility types are missing, and that some establishments have multiple facility types. Moreover, there seems to be too many facility types and some of them are misspelled. 

Let us first limit the number of facility type per establishment to 1 and correct spelling mistakes.

In [43]:
spell = Speller(lang='en')
inspections['Facility Type'] = inspections['Facility Type'].str.lower().apply(lambda x: spell(str(x))).apply(lambda x: re.sub(',|&|;','/',str(x)).split('/')[0])

Now let us see if one establishment can have multiple facility types for each inspection.

In [44]:
#we have to declare this function to be able to use it into aggregate
def to_list(a):
    return set(a)
establisment_facility_types = inspections[['DBA Name','Facility Type']].drop_duplicates().groupby('DBA Name')['Facility Type'].agg({'nbr_types': len, 'possible_types': to_list})
print("Possible number of different Facility Types for some establishment : ", establisment_facility_types['nbr_types'].unique())
establisment_facility_types[establisment_facility_types['nbr_types']>2].head()

Possible number of different Facility Types for some establishment :  [1 2 4 3 7 5 6 9]


,nbr_types,possible_types
DBA Name,,
7-ELEVEN,4,"{restaurant, grocery store, liquor, nan}"
ALASKA PALETERIA Y NEVERIA,3,"{restaurant, mobile frozen desserts dispenser-..."
ALL ABOUT KIDS LEARNING ACADEMY,3,"{children's services facility, nan, daycare co..."
ARAMARK,7,"{charter school, special event, northerly isla..."
ARAMARK EDUCATION SERVICES,3,"{restaurant, charter school, school}"


We can see that some of the establishments have two possible Facility types, in this case we replace the rows which have null value with one of the possible types.

In [45]:
#First remove NAN from sets
establisment_facility_types['possible_types'] = establisment_facility_types.apply(
    lambda row: {x for x in row['possible_types'] if pd.notna(x)},
    axis=1
) 
#Then keep only one type per establishment
establisment_facility_types['possible_types'] = establisment_facility_types.apply(
    lambda row: next(iter(row['possible_types'])) if len(row['possible_types'])!=0 else np.NaN,
    axis=1
) 
establisment_types_dict=establisment_facility_types['possible_types'].to_dict()
#Assign a type to each missing value
inspections['Facility Type'] = inspections.apply(
    lambda row: establisment_types_dict[row['DBA Name']] if pd.isna(row['Facility Type']) else row['Facility Type'],
    axis=1
)

In [46]:
print('{}% of entries still have missing facility types.'.format(100 * len(inspections[inspections['Facility Type'] == 'nan'])/len(inspections)))

2.444007535818237% of entries still have missing facility types.


We can now proceed to replace all missing facility types with 'other'.

In [47]:
inspections['Facility Type'] = inspections.apply(
    lambda row: 'other' if row['Facility Type'] == 'nan' else row['Facility Type'],
    axis=1
)

In [48]:
unique_types = pd.Series(inspections['Facility Type'].unique())
unique_types = pd.Series(unique_types.apply(lambda x: re.sub(',|&|;','/',str(x)).split('/')[0]).unique())#.explode().unique())
unique_types.apply(lambda x: str(x).strip())

0                        restaurant
1                             other
2                     grocery store
3      children's services facility
4                       coffee shop
                   ...             
347                    soup kitchen
348                    hooka lounge
349                       religious
350                wholesale bakery
351                      kids cafe'
Length: 352, dtype: object

There are 346 facility types in the dataframe. Let's see if we can cluster some of them.

In [49]:
school_vocab = ['school', 'under 6', 'university', 'cafeteria', 'training program', 'kids', 'children', 'daycare', 'years', 'school cafeteria', 'college', 'class', 'day care']
restaurant_vocab = ['restaurant', 'smokehouse', 'diner', 'breakfast', 'lunch', 'grill', 'sushi','banquet', 'dining', 'taqueria']
homes_vocab = ['long term care','assisted living', 'nursing', 'care', 'supportive']
religious_vocab = ['religious', 'religion', 'church', 'synaguogue', 'temple']

coffee_vocab = ['coffee', 'cafe']
catering_vocab = ['cater']
bakery_vocab = ['bake', 'patisserie', 'boulangerie']
market_vocab = ['nutrition store', 'grocery', 'french market space', 'dollar store', 'butcher', 'deli', 'farmer\'s market', 'packaged health foods', 'fish market', 'meat market']

kiosk_vocab = ['butcher', 'snack', 'dollar', 'grocery', 'frozen food storage', 'meat packing', 'food', 'market', 'packaged', 'popcorn', 'pantry', 'store', 'produce', 'kiosk', 'convenience', 'commiasary','gas station', 'wholesale', 'deli', 'convenient', 'retail', 'dollar tree']
dessert_vocab = ['ice cream', 'dessert', 'paleteria', 'candy', 'gelato', 'donut']
health_vocab = ['gym', 'exercise', 'juice and salad bar', 'nutrition','herbal', 'fitness', 'drug', 'rehab', 'herbalife', 'weight', 'herb', 'health', 'decream']
medical_vocab = ['pharmacy']

vending_vocab = ['vend', 'pop-up', 'mobile', 'cart', 'dispenser', 'truck']
drinks_vocab = ['liquor', 'music', 'bar', 'pub', 'beverage', 'club', 'roof', 'tavern', 'brewery', 'wine', 'beer', 'lounge', 'tea', 'shakes']
live_vocab =['live', 'poultry', 'slaughter', 'farm']
kitchen_vocab = ['kitchen']

distributor_vocab = ['distributor', 'distribution']
shelter_vocab = ['shelter', 'youth housing']
# other_vocab = ['warehouse', 'other', 'theater', 'golf', 'laundromat', 'hotel', 'riverbank', 'theatre', 'event', 'special', 'museum', 'hospital', 'pool', 'art', 'airport', 'gallery', 'terminal']


In [50]:
def extract_label(unique_labels, vocab,mssg):
    labels = unique_labels.apply(lambda x: x if bool(re.search('|'.join(vocab), x)) else np.nan).dropna()
    print(labels)
    print('{}: {}'.format(mssg, labels.size))
    unique_labels = unique_labels.drop(labels=labels.index,errors='ignore')
    return unique_labels, labels

In [51]:
unique_types, school_labels = extract_label(unique_types, school_vocab, 'Number of Schools: ')
unique_types, restaurant_labels = extract_label(unique_types, restaurant_vocab, 'Number of Restaurants: ')
unique_types, homes_labels = extract_label(unique_types, homes_vocab, 'Number of Nursing homes: ')
unique_types, religious_labels = extract_label(unique_types, religious_vocab, 'Number of Religious Establishments: ')

unique_types, coffee_labels = extract_label(unique_types, coffee_vocab, 'Number of Coffeeshops: ')
unique_types, catering_labels = extract_label(unique_types, catering_vocab, 'Number of Caterers: ')
unique_types, bakery_labels = extract_label(unique_types, bakery_vocab, 'Number of Bakeries: ')
unique_types, market_labels = extract_label(unique_types, market_vocab, 'Number of Markets: ')

unique_types, kiosk_labels = extract_label(unique_types, kiosk_vocab, 'Number of Kiosks: ')
unique_types, dessert_labels = extract_label(unique_types, dessert_vocab, 'Number of Dessert Places: ')
unique_types, health_labels = extract_label(unique_types, health_vocab, 'Number of Health Institutions: ')
unique_types, medical_labels = extract_label(unique_types, medical_vocab, 'Number of Pharmacies: ')

unique_types, vending_labels = extract_label(unique_types, vending_vocab, 'Number of Vending Establishments: ')
unique_types, drinks_labels = extract_label(unique_types, drinks_vocab, 'Number of Drinks Places: ')
unique_types, live_labels = extract_label(unique_types, live_vocab, 'Number of Live Animal Sellers and Slaughterhouses: ')
unique_types, kitchen_labels = extract_label(unique_types, kitchen_vocab, 'Number of Shared Kitchen: ')

unique_types, distributor_labels = extract_label(unique_types, distributor_vocab, 'Number of Distributors: ')
unique_types, shelter_labels = extract_label(unique_types, shelter_vocab, 'Number of Shelters: ')
other_labels = unique_types


3                children's services facility
7                       daycare (2 - 6 years)
8             daycare above and under 2 years
19                                     school
21                   15 months to 5 years old
22                         daycare combo 1586
26                    daycare (under 2 years)
28                      day care combo (1586)
29                             charter school
30         1023 children's service s facility
38                            teaching school
57                 children services facility
58                          daycare (2 years)
64                children's service facility
69          1023 children's services facility
70                             cooking school
71                             private school
79                              daycare night
80                       after school program
88                                    daycare
108                           senior day care
115         1023-children's servic

In [52]:
def get_dict_from_list(labels, val):
    return {x: val for x in labels}

In [53]:
dict_gen_types = dict()

school_dict = get_dict_from_list(school_labels,'Educational Establishment')
dict_gen_types.update(school_dict)
restaurant_dict = get_dict_from_list(restaurant_labels,'Restaurant')
dict_gen_types.update(restaurant_dict)
homes_dict = get_dict_from_list(homes_labels,'Retirement Homes')
dict_gen_types.update(homes_dict)
religious_dict = get_dict_from_list(religious_labels,'Religious Establishment')
dict_gen_types.update(religious_dict)

coffee_dict = get_dict_from_list(coffee_labels,'Coffeeshop')
dict_gen_types.update(coffee_dict)
catering_dict = get_dict_from_list(catering_labels,'Catering Establishment')
dict_gen_types.update(catering_dict)
bakery_dict = get_dict_from_list(bakery_labels,'Bakery')
dict_gen_types.update(bakery_dict)
market_dict = get_dict_from_list(market_labels,'Market')
dict_gen_types.update(market_dict)

kiosk_dict = get_dict_from_list(kiosk_labels,'Kiosk')
dict_gen_types.update(kiosk_dict)
dessert_dict = get_dict_from_list(dessert_labels,'Dessert Establishment')
dict_gen_types.update(dessert_dict)
health_dict = get_dict_from_list(health_labels,'Health Establishment')
dict_gen_types.update(health_dict)
medical_dict = get_dict_from_list(medical_labels,'Medical Establishment')
dict_gen_types.update(medical_dict)

vending_dict = get_dict_from_list(vending_labels,'Vending')
dict_gen_types.update(vending_dict)
drinks_dict = get_dict_from_list(drinks_labels,'Drinks Establishment')
dict_gen_types.update(drinks_dict)
live_dict = get_dict_from_list(live_labels,'Live Animals')
dict_gen_types.update(live_dict)
kitchen_dict = get_dict_from_list(kitchen_labels,'Shared Kitchens')
dict_gen_types.update(kitchen_dict)

distributor_dict = get_dict_from_list(distributor_labels,'Distributors')
dict_gen_types.update(distributor_dict)
shelter_dict = get_dict_from_list(shelter_labels,'Shelters')
dict_gen_types.update(shelter_dict)
other_dict = get_dict_from_list(other_labels,'Other')
dict_gen_types.update(other_dict)


In [54]:
inspections['Facility Type'] = inspections['Facility Type'].replace(dict_gen_types)

In [55]:
inspections['Facility Type'].unique()

array(['Restaurant', 'Other', 'Market', 'Educational Establishment',
       'Coffeeshop', 'Drinks Establishment', 'Catering Establishment',
       'Bakery', 'Kiosk', 'Vending', 'Live Animals', 'Retirement Homes',
       'Shared Kitchens', 'Dessert Establishment', 'Health Establishment',
       'Shelters', 'Religious Establishment', 'Medical Establishment',
       'Distributors'], dtype=object)

We have finally cleaned the facility type.

### F. Clean the AKA names column

We know that the AKA name is the name of the restaurant as known to the public. We decide to duplicate the DBA name to the missing AKA names: We will need those for our recommendation map later on, and it make more sense to display the AKA names for the users.

In [56]:
inspections['AKA Name'].fillna(inspections['DBA Name'], inplace=True)

### G. Explore the violations column

Let's first see if the missing violation are consistent with our entries

In [57]:
inspections[inspections['Violations'].isnull()]['Results'].unique()

array(['Pass', 'Out of Business', 'Not Ready', 'Fail',
       'Pass w/ Conditions', 'No Entry', 'Business Not Located'],
      dtype=object)

We expected the entries which have null Violations to have 'No Entry', 'Out of Business', 'Pass' or 'Business Not Located' as a value for Results. We see that we alse have null Violations for 'Fail' and 'Pass w/ Conditions' inspections. We still keep those entries because they can be useful for other metrics.

Let us now inspect the Violations column and try to make sense of what it contains.

In [58]:
inspections['Violations']

0         3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...
1         3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...
2         3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...
3                                                       NaN
4         16. FOOD PROTECTED DURING STORAGE, PREPARATION...
                                ...                        
195317                                                  NaN
195318    33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...
195319    33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...
195320    33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...
195321                                                  NaN
Name: Violations, Length: 194803, dtype: object

They all seem to have a specific structure. Let's examine one closely to look at how a Violation entry is structured.

In [59]:
inspections['Violations'][0]

'3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL EMPLOYEE; KNOWLEDGE, RESPONSIBILITIES AND REPORTING - Comments: 2-102.14(O)  OBSERVED NO WRITTEN EMPLOYEE HEALTH POLICY ON THE PREMISES. MANAGEMENT INSTRUCTED TO HAVE A SIGNED EMPLOYEE HEALTH POLICY FOR EACH EMPLOYEE. PRIORITY FOUNDATION 7-38-010. NO CITATION ISSUED. | 5. PROCEDURES FOR RESPONDING TO VOMITING AND DIARRHEAL EVENTS - Comments: 2-501.11  OBSERVED NO CLEAN UP PROCEDURE AND SUPPLIES ON THE PREMISES TO RESPOND TO VOMIT AND DIARRHEAL EVENTS. MANAGEMENT INSTRUCTED TO PROVIDE BOTH. PRIORITY FOUNDATION 7-38-005. NO CITATION ISSUED. | 58. ALLERGEN TRAINING AS REQUIRED - Comments: 2-102.13  OBSERVED THE CERTIFIED FOOD MANAGER WITHOUT AN ALLERGEN TRAINING CERTIFICATE. MANAGEMENT INSTRUCTED THAT ALL CERTIFIED FOOD MANAGERS MUST SHOW PROOF OF TRAINING.'

We can see that violations are separated by '|'. Each one begins with the violation number and type, followed by a sentence as to why this violation was found in the establishment and a sentence describing what the management needs to do. Let us hence try and have a column for each feature given by the Violations entry.

In [60]:
def violations_analysis(violations):
    if(violations == ['']):
        return(np.nan, np.nan)
    nb_violations = []
    comments_violations = []
    for violation in violations:
        nb_violations.append(int(violation.split('.')[0]))
        split = violation.split(':')
#         print(violation)
        if(len(split) != 1):
            comments_violations.append(violation.split(':')[1])
    return (nb_violations, comments_violations)

In [61]:
exploded_violations = inspections['Violations'].replace({np.nan: ''}).str.split('|').apply(violations_analysis)#.replace({np.nan:''}).apply(violations_analysis)


In [62]:
inspections[['Violation Numbers', 'Violation Comments']] = pd.DataFrame(exploded_violations.tolist())    


In [63]:
inspections.head()

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location,Community Area,Violation Numbers,Violation Comments
0,2345318,SUBWAY,SUBWAY,2529116.0,Restaurant,Risk 1 (High),2620 N NARRAGANSETT AVE,2019-11-08T00:00:00.000,Canvass Re-Inspection,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.927995,-87.785752,"41.92799528871574, -87.78575236468352",BELMONT CRAGIN,"[3, 5, 58]",[ 2-102.14(O) OBSERVED NO WRITTEN EMPLOYEE HE...
1,2321166,SUBWAY,SUBWAY,2529116.0,Restaurant,Risk 1 (High),2620 N NARRAGANSETT AVE,2019-11-06T00:00:00.000,Canvass,Fail,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.927995,-87.785752,"41.92799528871574, -87.78575236468352",BELMONT CRAGIN,"[3, 5, 10, 22, 33, 57, 58]",[ OBSERVED NO WRITTEN EMPLOYEE HEALTH POLICY O...
2,2290717,PAPA JOHN'S PIZZA,PAPA JOHN'S PIZZA,1545897.0,Restaurant,Risk 2 (Medium),2620 N NARRAGANSETT AVE,2019-05-30T00:00:00.000,Canvass,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.927995,-87.785752,"41.92799528871574, -87.78575236468352",BELMONT CRAGIN,"[3, 5]",[ OBSERVED NO EMPLOYEE HEALTH POLICY ON THE PR...
3,2144892,SUBWAY,SUBWAY,2529116.0,Restaurant,Risk 1 (High),2620 N NARRAGANSETT AVE,2018-02-13T00:00:00.000,Canvass Re-Inspection,Pass,NaN,41.927995,-87.785752,"41.92799528871574, -87.78575236468352",BELMONT CRAGIN,NaN,NaN
4,2144588,SUBWAY,SUBWAY,2529116.0,Restaurant,Risk 1 (High),2620 N NARRAGANSETT AVE,2018-02-06T00:00:00.000,Canvass,Fail,"16. FOOD PROTECTED DURING STORAGE, PREPARATION...",41.927995,-87.785752,"41.92799528871574, -87.78575236468352",BELMONT CRAGIN,"[16, 34, 42]",[ OBSERVED BLACK AND PINK MOLD LIKE SUBSTANCES...


In [64]:
print('There are still {}% rows missing their violations info.'.format(100 * inspections['Violation Numbers'].isna().sum()/len(inspections)))

There are still 26.71724768099054% rows missing their violations info.


We can see that there is still missing data but we have no choice but to keep it as we cannot drop 29% of the dataset.

In [65]:
inspections.to_pickle('./tmp_inspections')

In [66]:
#TODO: Create a pickle of all possible violations with their respective numbers

### H. Cleaning the Inspection Type

Let's see what's left to be done in our data cleaning process regarding the inspection type.

In [67]:
print('{}% of rows are missing the Inspection Type entry.'.format(100 * inspections['Inspection Type'].isna().sum()/len(inspections['Inspection Type'])))

0.000513339116954051% of rows are missing the Inspection Type entry.


As we can see the number of entries which have missing Inspection Type is really small, we can safely drop those.

In [68]:
inspections.dropna(axis=0, subset=['Inspection Type'], inplace=True)

First, let us reformat the inspection types and collect all unique ones to examine them.

In [69]:
inspections['Inspection Type'] = inspections['Inspection Type'].str.lower().apply(lambda x: spell(str(x)))

In [70]:
inspections['Inspection Type'].unique()

array(['canvass re-inspection', 'canvass', 'license',
       'short form complaint', 'complaint', 'license re-inspection',
       'license-task force', 'complaint re-inspection', 'out of business',
       'suspected food poisoning re-inspection',
       'suspected food poisoning', 'recent inspection', 'consultation',
       'complaint-fire', 'tag removal', 'illegal operation',
       'package liquor 1474', 'short form fire-complaint',
       'special events (festivals)', 'complaint-fire re-inspection',
       'task force liquor 1475', 'not ready', 'task force liquor 1470',
       'license request', 'no entry', 'non-inspection', 'possible fbi',
       'pre-license consultation', 'summer feeding', 'expansion',
       'error save', 'taskforce', '1315 license reinspection',
       'canvass for rib fest', 'close-up/complaint reinspection',
       'sip/complaint', 'sip', 'finish complaint inspection from 5-18-10',
       'special task force', 'canvass re inspection of close up',
       'reca

From just observing the data above (as well as using the file describing the different columns in the dataset provided by the City of Chicago), we notice the following:

* Some inspections are canvass related. These correspond to regular inspections, performed at a frequency relative to the risk of the establishment.
* Some inspections are consultation related. These are the ones done at the request of the owner before th eopening of an establishment.
* Some inspections are license related such as license renewal and license request.
* Soms inspections are complaint based such as complaint and complain re-inspection.
* Some inspections are done following a suspected food poisoning
* Some inspections are specially for alcohol related areas such as liquor task force and liquor catering.
* Some inspections are to make sure some changes have been implemented such as recall or reinspection after 48 hour notice.
* All of the above can either be inspections or reinspections

Using this information, let's try to group together similar inspection types. Let us also add a column specifying whether the inspection is a reinspection or not.
To that end, we use our previously defined functions, extract_label and get_dict_from_list.

In [71]:
unique_insp_types = pd.Series(inspections['Inspection Type'].unique())

Let us now proceed with categorizing them into reinspections and first-time inspections.

In [72]:
re_inspection_vocab = ['recall', 'renewal', 'recent inspection', 'corrective action', 'reinspection', 're-inspection']

In [73]:
first_timers, repeated_insp = extract_label(unique_insp_types, re_inspection_vocab, 'Reinspections')

0                      canvass re-inspection
5                      license re-inspection
7                    complaint re-inspection
9     suspected food poisoning re-inspection
11                         recent inspection
19              complaint-fire re-inspection
32                 1315 license reinspection
34           close-up/complaint reinspection
40                         recall inspection
41                         corrective action
53                              reinspection
59    license renewal inspection for daycare
65                  day care license renewal
68            reinspection of 48 hour notice
79               license renewal for daycare
83                 re-inspection of close-up
dtype: object
Reinspections: 16


In [74]:
dict_reinspections = get_dict_from_list(first_timers,0)
dict_reinspections.update(get_dict_from_list(repeated_insp,1))


In [75]:
inspections['Reinspections'] = inspections['Inspection Type'].replace(dict_reinspections)

In [76]:
inspections['Reinspections'].unique()

array([1, 0])

We have successfully added the feature 'Reinspection'.

We may now proceed with categorizing the inspection types into categories according to their type.

In [77]:
canvass_vocab = ['canvass', 'canvas']
consultation_vocab = ['consultation']
license_vocab = ['license']
complains_vocab = ['complain', 'harp', 'fire']
poison_vocab = ['poison', 'sick']
task_force_vocab = ['force','task', 'liquor', 'tavern']
# Note: difference between this and reinspections is: this is exclusively for reinspection after a failed first inspection or a notice
recall_vocab = ['recall', 'recent inspection', 'corrective action' 'reinspection', 'reinspection of 48 hour notice', 're-inspection of close-up']


In [78]:
unique_insp_types, canvass = extract_label(unique_insp_types, canvass_vocab, 'Canvass')
unique_insp_types, consultation = extract_label(unique_insp_types, consultation_vocab, 'Consultations')
unique_insp_types, license = extract_label(unique_insp_types, license_vocab, 'Licenses')
unique_insp_types, complaints = extract_label(unique_insp_types, complains_vocab, 'Complaints')
unique_insp_types, poisonings = extract_label(unique_insp_types, poison_vocab, 'Poisonings')
unique_insp_types, task_forces = extract_label(unique_insp_types, task_force_vocab, 'Task-Forces')
unique_insp_types, recalls = extract_label(unique_insp_types, recall_vocab, 'Recalls')


0                 canvass re-inspection
1                               canvass
33                 canvass for rib fest
39    canvass re inspection of close up
46                               canvas
49         canvass school/special event
54                canvass/special event
84               canvass special events
dtype: object
Canvass: 8
12                consultation
27    pre-license consultation
85        license consultation
dtype: object
Consultations: 3
2                                       license
5                         license re-inspection
6                            license-task force
23                              license request
32                    1315 license reinspection
52                         license daycare 1586
59       license renewal inspection for daycare
63                            license/not ready
65                     day care license renewal
67    license task force / not -for-profit club
70                    license canceled by owner
74 

Let's see if there are any inspection types left that have not been grouped.

In [79]:
unique_insp_types.size

28

In [80]:
unique_insp_types

8                   out of business
14                      tag removal
15                illegal operation
18       special events (festivals)
21                        not ready
24                         no entry
25                   non-inspection
26                     possible fbi
28                   summer feeding
29                        expansion
30                       error save
36                              sip
41                corrective action
45               changed court date
48                         addendum
50                 taste of chicago
53                     reinspection
55           sip recently inspected
60    kitchen closed for renovation
64                       duplicated
66                sample collection
71               citation re-issued
77                     out business
82                             o.b.
90                             city
91                        kids cafe
93             business not located
94                       kid

By looking at them, we can see that:

* Most of them don't make sense such as kids cafe, corrective action and possible fbi which we can group together in an 'other' category.
* Some of them are closed, out of business or the inspection has simply not been conducted so they can be put together in a 'non-inspection' category.

In [81]:
no_insp_vocab = ['closed', 'o.b.', 'out business', 'out of business', 'non-inspection', 'no entry', 'not ready', 'not located']

In [82]:
unique_insp_types, no_insp = extract_label(unique_insp_types, no_insp_vocab, 'No Inspection')
other = unique_insp_types

8                   out of business
21                        not ready
24                         no entry
25                   non-inspection
60    kitchen closed for renovation
77                     out business
82                             o.b.
93             business not located
dtype: object
No Inspection: 8


In [83]:
dict_insp_type = {}

dict_canvass = get_dict_from_list(canvass,'Canvass')
dict_insp_type.update(dict_canvass)
dict_consultation = get_dict_from_list(consultation,'Consultation')
dict_insp_type.update(dict_consultation)
dict_license = get_dict_from_list(license,'License')
dict_insp_type.update(dict_license)

dict_complaints = get_dict_from_list(complaints,'Complaint')
dict_insp_type.update(dict_complaints)
dict_poisonings = get_dict_from_list(poisonings,'Poisoning')
dict_insp_type.update(dict_poisonings)
dict_task_forces = get_dict_from_list(task_forces,'Task-Forces')
dict_insp_type.update(dict_task_forces)

dict_recalls = get_dict_from_list(recalls,'Recall')
dict_insp_type.update(dict_recalls)
dict_no_insp = get_dict_from_list(no_insp,'No Inspection')
dict_insp_type.update(dict_no_insp)
dict_other = get_dict_from_list(other,'Other')
dict_insp_type.update(dict_other)

In [84]:
inspections['Inspection Type'] = inspections['Inspection Type'].replace(dict_insp_type)

In [85]:
inspections['Inspection Type'].unique()

array(['Canvass', 'License', 'Complaint', 'No Inspection', 'Poisoning',
       'Recall', 'Consultation', 'Other', 'Task-Forces'], dtype=object)

### I. Clean Risk column

Let's look at the unique values the risk can take.

In [86]:
inspections['Risk'].unique()

array(['Risk 1 (High)', 'Risk 2 (Medium)', 'Risk 3 (Low)', nan, 'All'],
      dtype=object)

First of all, let's extract the Risk factor from the string, that way we will be able to perform arithmetic comparisons useful in our future analysis.

E.g. 'Risk 3 (Low)' will be mapped 3

In [87]:
def find_risk(entry):
    if (entry == 'All'):
        return 3
    elif(pd.notna(entry)):
        return int(re.findall('\d+', entry)[0])
    else:
        return np.nan

inspections['Risk'] = inspections['Risk'].apply(find_risk)

In [88]:
inspections['Risk'].unique()

array([ 1.,  2.,  3., nan])

Let us look at how many missing values we have.

In [89]:
print('{}% of rows are missing the Risk entry.'.format(100 * inspections['Risk'].isna().sum()/len(inspections['Risk'])))


0.036447264401802855% of rows are missing the Risk entry.


We can safely drop missing values since they represent only 0.036%. **CAN WE PLEASE?**

In [90]:
# inspections.dropna(axis=0, subset=['Risk'], inplace=True)

In [91]:
# print('{}% of rows are missing the Risk entry.'.format(100 * inspections['Risk'].isna().sum()/len(inspections['Risk'])))

Now we will try to recover the missing values from other data entries: we base our observation on the fact that it makes more sence to attribute the lowest value (which corresponds to the highest risk) the establishment has been attributed if it had several possible values in the past.

In [92]:
print("Number of null Risk values before recovering: ", len(inspections[inspections['Risk'].isnull()]))
inspections[inspections['Risk'].isnull()].head(2)

Number of null Risk values before recovering:  71


,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location,Community Area,Violation Numbers,Violation Comments,Reinspections
3468,2321069,MJ ONE STOP SHOP,MJ ONE STOP SHOP,2698807.0,Other,NaN,249 E 115TH ST,2019-11-05T00:00:00.000,License,No Entry,NaN,41.685285,-87.616069,"41.6852846288262, -87.61606883053517",WEST PULLMAN,NaN,NaN,0
11720,2316086,UNIVERSITY CENTER PROPERTY LLC,UNIVERSITY CENTER PROPERTY LLC,2694616.0,Other,NaN,525 S STATE ST,2019-10-18T00:00:00.000,License,Not Ready,NaN,41.875179,-87.627474,"41.875179056057796, -87.6274737894488",LOOP,"[33, 34, 36]",[ CLEAN SURFACES UNDER DISPLAY COOLERS IN FRON...,0


In [93]:
#we have to declare this function to be able to use it into aggregate
def to_set(a):
    #return {x for x in a if pd.notna(x)}
    return set(a)
establisment_risks = inspections[['DBA Name','Risk']].drop_duplicates().groupby('DBA Name')['Risk'].agg({'nbr_values': len, 'possible_values': to_set})
print("Possible number of values for risk for some establishment : ", establisment_risks['nbr_values'].unique())
establisment_risks[establisment_risks['nbr_values']>1].head()

Possible number of values for risk for some establishment :  [1. 2. 3. 4.]


,nbr_values,possible_values
DBA Name,,
11 DINING,2.0,"{1.0, 2.0}"
"11 DINING, LLC",3.0,"{1.0, 2.0, 3.0}"
111 COFFEE BAR,2.0,"{2.0, 3.0}"
123 FOOD,2.0,"{2.0, 3.0}"
"14 W. HUBBARD, LLC",2.0,"{1.0, 3.0}"


We see that the same establishment may have different Risk values (maybe the latter varies in time depending on the result of inspections). We choose to fill in the missing values in the Risk column by taking the minimum value of risk for that establishment.

In [94]:
#First remove NAN from sets
establisment_risks['possible_values'] = establisment_risks.apply(
    lambda row: {int(x) for x in row['possible_values'] if pd.notna(x)},
    axis=1
) 
#Then keep only one type per establishment
establisment_risks['possible_values'] = establisment_risks.apply(
    lambda row: min(row['possible_values']) if len(row['possible_values'])!=0 else np.NaN,
    axis=1
) 
establisment_risks_dict=establisment_risks['possible_values'].to_dict()
#Assign a type to each missing value
inspections['Risk'] = inspections.apply(
    lambda row: establisment_risks_dict[row['DBA Name']] if pd.isna(row['Risk']) else row['Risk'],
    axis=1
)

In [95]:
print("Number of null Risk values after recovering: ",len(inspections[inspections['Risk'].isnull()]))
inspections[inspections['Risk'].isnull()].head(2)

Number of null Risk values after recovering:  45


,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location,Community Area,Violation Numbers,Violation Comments,Reinspections
3468,2321069,MJ ONE STOP SHOP,MJ ONE STOP SHOP,2698807.0,Other,NaN,249 E 115TH ST,2019-11-05T00:00:00.000,License,No Entry,NaN,41.685285,-87.616069,"41.6852846288262, -87.61606883053517",WEST PULLMAN,NaN,NaN,0
11720,2316086,UNIVERSITY CENTER PROPERTY LLC,UNIVERSITY CENTER PROPERTY LLC,2694616.0,Other,NaN,525 S STATE ST,2019-10-18T00:00:00.000,License,Not Ready,NaN,41.875179,-87.627474,"41.875179056057796, -87.6274737894488",LOOP,"[33, 34, 36]",[ CLEAN SURFACES UNDER DISPLAY COOLERS IN FRON...,0


We see that we were able to recover 42% of the missing values; for the others let's use the default value 'Risk 2 (Medium)' as it's the medium value.

In [96]:
inspections['Risk'].fillna('2', inplace = True)
print("Number of null Risk values after filling: ",len(inspections[inspections['Risk'].isnull()]))

Number of null Risk values after filling:  0


### J. Clean the License# column

Let's look at what values the license number may take.

In [97]:
inspections['License #'].unique()

array([2529116., 1545897., 1621730., ...,   14103.,   61801., 2009488.])

As expected, they are integers. Let's see if there are any missing values and if so, how many.

In [98]:
print('{}% of rows are missing the License # entry.'.format(100 * inspections['License #'].isna().sum()/len(inspections['License #'])))

0.008726809786347163% of rows are missing the License # entry.


What do the entries with no "license #" look like?

In [99]:
inspections[inspections['License #'].isnull()]

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location,Community Area,Violation Numbers,Violation Comments,Reinspections
74120,2290863,ST. DEMETRIOS GREEK ORTHODOX CHURCH,ST. DEMETRIOS CHURCH,NaN,Other,2,2727 W WINONA ST,2019-06-04T00:00:00.000,Canvass,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.974653,-87.697529,"41.974653353169366, -87.69752945714045",LINCOLN SQUARE,"[33, 34, 35, 36, 38]","[ MUST CLEAN DRIED FOOD DEBRIS FROM SLICER, LO...",0
74121,2181316,ST. DEMETRIOS GREEK ORTHODOX CHURCH,ST. DEMETRIOS CHURCH,NaN,Other,2,2727 W WINONA ST,2018-06-13T00:00:00.000,Canvass,Pass,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,41.974653,-87.697529,"41.974653353169366, -87.69752945714045",LINCOLN SQUARE,"[16, 33, 34, 35, 36, 38]",[ OBSERVED A BLACK SLIME LIKE SUBSTANCE ON THE...,0
74122,2071910,ST. DEMETRIOS GREEK ORTHODOX CHURCH,ST. DEMETRIOS CHURCH,NaN,Other,2,2727 W WINONA ST,2017-08-04T00:00:00.000,Canvass,Pass,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,41.974653,-87.697529,"41.974653353169366, -87.69752945714045",LINCOLN SQUARE,"[34, 38]","[ CONSTRUCTED PER CODE, CLEANED, GOOD REPAIR, ...",0
74123,1933084,ST. DEMETRIOS GREEK ORTHODOX CHURCH,ST. DEMETRIOS CHURCH,NaN,Other,2,2727 W WINONA ST,2016-06-20T00:00:00.000,Canvass,Pass,38. VENTILATION: ROOMS AND EQUIPMENT VENTED AS...,41.974653,-87.697529,"41.974653353169366, -87.69752945714045",LINCOLN SQUARE,"[16, 29, 32, 33, 34, 35, 38, 40, 43]",[ OBSERVED INTERIOR PANELS OF ICE MACHINE IN T...,0
74124,1561809,ST. DEMETRIOS GREEK ORTHODOX CHURCH,ST. DEMETRIOS CHURCH,NaN,Other,2,2727 W WINONA ST,2015-08-04T00:00:00.000,Canvass,Pass,NaN,41.974653,-87.697529,"41.974653353169366, -87.69752945714045",LINCOLN SQUARE,"[31, 32, 33, 34, 35, 36, 38, 40, 41, 43, 45]",[ NO REUSE OF SINGLE SERVICE ARTICLES - Commen...,0
74125,1459918,ST. DEMETRIOS GREEK ORTHODOX CHURCH,ST. DEMETRIOS CHURCH,NaN,Other,2,2727 W WINONA ST,2014-05-20T00:00:00.000,Canvass,Pass,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,41.974653,-87.697529,"41.974653353169366, -87.69752945714045",LINCOLN SQUARE,"[14, 29, 30, 31, 32, 33, 34, 35, 36, 38, 40, 4...",[ PREVIOUS SERIOUS VIOLATION FROM REPORT #1764...,0
74126,1099104,ST. DEMETRIOS GREEK ORTHODOX CHURCH,ST. DEMETRIOS CHURCH,NaN,Other,2,2727 W WINONA ST,2013-07-24T00:00:00.000,Canvass,Pass,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,41.974653,-87.697529,"41.974653353169366, -87.69752945714045",LINCOLN SQUARE,"[8, 19, 29, 30, 31, 32, 33, 34, 35, 36, 38, 40...","[ CLEAN, PROPER TEMPERATURE, CONCENTRATION, E...",0
74127,1188285,ST. DEMETRIOS GREEK ORTHODOX CHURCH,ST. DEMETRIOS CHURCH,NaN,Other,2,2727 W WINONA ST,2012-07-25T00:00:00.000,Canvass,Pass,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,41.974653,-87.697529,"41.974653353169366, -87.69752945714045",LINCOLN SQUARE,"[32, 33, 35, 38, 41, 43, 45]",[ OBSERVED ICE MACHINE LID IN DISREPAIR. ICE M...,0
74128,521659,ST. DEMETRIOS GREEK ORTHODOX CHURCH,ST. DEMETRIOS CHURCH,NaN,Other,2,2727 W WINONA ST,2011-08-10T00:00:00.000,Canvass,Pass,NaN,41.974653,-87.697529,"41.974653353169366, -87.69752945714045",LINCOLN SQUARE,"[18, 21, 32, 32, 33, 33, 35, 38, 41, 41, 41, 4...",[ OBSERVED NO UP TO DATE PEST CONTROL LOG BOOK...,0
74129,339207,ST DEMETRIOS CHURCH,ST DEMETRIOS CHURCH,NaN,Other,1,2727 W WINONA ST,2010-07-30T00:00:00.000,Other,Pass,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,41.974653,-87.697529,"41.974653353169366, -87.69752945714045",LINCOLN SQUARE,"[30, 33, 34, 35, 38]",[ CUSTOMER ADVISORY POSTED AS NEEDED - Comment...,0


In [100]:
inspections[inspections['DBA Name']=='ARGENTINA FOODS']

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location,Community Area,Violation Numbers,Violation Comments,Reinspections
161311,1152076,ARGENTINA FOODS,ARGENTINA FOODS,NaN,Market,2,4500 S WOOD ST,2014-04-10T00:00:00.000,Canvass,Out of Business,NaN,41.812105,-87.670072,"41.812105152977246, -87.67007183351623",WEST ENGLEWOOD,"[32, 33, 34, 35, 36, 38, 41]",[ OBSERVED RAW WOOD PANELING ATTACHED TO WALL ...,0
161314,158274,ARGENTINA FOODS,ARGENTINA FOODS,57047.0,Market,2,4500 S WOOD ST,2010-01-20T00:00:00.000,No Inspection,Fail,NaN,41.812105,-87.670072,"41.812105152977246, -87.67007183351623",WEST ENGLEWOOD,"[3, 30, 32, 33, 34, 35, 41]",[ OBSERVED IMPROPER TEMPERATURES OF POTENTIALL...,0
161315,158273,ARGENTINA FOODS,ARGENTINA FOODS,57047.0,Market,2,4500 S WOOD ST,2010-01-20T00:00:00.000,No Inspection,Fail,NaN,41.812105,-87.670072,"41.812105152977246, -87.67007183351623",WEST ENGLEWOOD,"[6, 32, 33, 34, 35, 36, 38, 40]",[ OBSERVED HAND WASHING FACILITIES FOR STAFF N...,0


* We see that almost all of the missing License numbers are those of churches (Maybe they don't need a lisence to serve food). We assign the default license number 00000 to those entries.
* The only real license numbers missing is the one from ARGENTINA FOODS and we are able to assign it manually.

In [101]:
inspections.loc[159539,('License #')]='57047'
inspections['License #'].fillna('00000',inplace=True)

In [102]:
print('{}% of rows are missing the License # entry.'.format(100 * inspections['License #'].isna().sum()/len(inspections['License #'])))

0.0% of rows are missing the License # entry.


### K. Clean the Date column

Let's examine the date column.

In [103]:
print('{}% of rows are missing the Inspection Date entry.'.format(100 * inspections['Inspection Date'].isna().sum()/len(inspections['Inspection Date'])))


0.0% of rows are missing the Inspection Date entry.


In [104]:
inspections['Inspection Date'].unique()

array(['2019-11-08T00:00:00.000', '2019-11-06T00:00:00.000',
       '2019-05-30T00:00:00.000', ..., '2012-02-22T00:00:00.000',
       '2010-04-04T00:00:00.000', '2010-02-20T00:00:00.000'], dtype=object)

It seems to contain strings. Let us convert it to a datetime object.

In [105]:
inspections['Inspection Date'] = inspections['Inspection Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%f"))

In [106]:
inspections['Inspection Date'].head()

0   2019-11-08
1   2019-11-06
2   2019-05-30
3   2018-02-13
4   2018-02-06
Name: Inspection Date, dtype: datetime64[ns]

### L. Examine the Results column

In [107]:
print('{}% of rows are missing the Results entry.'.format(100 * inspections['Results'].isna().sum()/len(inspections['Results'])))


0.0% of rows are missing the Results entry.


In [108]:
inspections['Results'].unique()

array(['Pass w/ Conditions', 'Fail', 'Pass', 'Out of Business',
       'Not Ready', 'No Entry', 'Business Not Located'], dtype=object)

It seems to be clean!

### M. Attribute the right types to each column

In [109]:
inspections.isna().sum().apply(lambda x: '{}% missing values'.format(100 * x/len(inspections)))

Inspection ID                        0.0% missing values
DBA Name                             0.0% missing values
AKA Name                             0.0% missing values
License #                            0.0% missing values
Facility Type                        0.0% missing values
Risk                                 0.0% missing values
Address                              0.0% missing values
Inspection Date                      0.0% missing values
Inspection Type                      0.0% missing values
Results                              0.0% missing values
Violations             26.51974825720475% missing values
Latitude                             0.0% missing values
Longitude                            0.0% missing values
Location                             0.0% missing values
Community Area                       0.0% missing values
Violation Numbers     26.717384831777906% missing values
Violation Comments    26.717384831777906% missing values
Reinspections                  

Now that we are done with replacing NaN values, we want to attribute the right type to each column. [documentation about categorical type](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html)


In [110]:
inspections['Inspection Date'].dt

In [111]:
inspections = inspections.astype({"Inspection ID": int, "DBA Name": str, "AKA Name": str, "License #": int,\
                                  "Facility Type": str, "Risk": int,"Address": str, "Inspection Type": str,\
                                   "Results": str, "Community Area": str})

We notice that the 'Inspection date' columns only contains dates and no times (the time seems to always be midnight by default). Hence we only keep the date and clean the values

In [112]:
inspections.loc[:,('Inspection Date')]=inspections.loc[:,('Inspection Date')].apply(pd.to_datetime)

In [113]:
inspections.dtypes

Inspection ID                  int64
DBA Name                      object
AKA Name                      object
License #                      int64
Facility Type                 object
Risk                           int64
Address                       object
Inspection Date       datetime64[ns]
Inspection Type               object
Results                       object
Violations                    object
Latitude                     float64
Longitude                    float64
Location                      object
Community Area                object
Violation Numbers             object
Violation Comments            object
Reinspections                  int64
dtype: object

In [114]:
inspections.to_pickle('./datasets/cleaned_inspections.pickle')